<a href="https://colab.research.google.com/github/JulianotCosta/Dictionary_IA/blob/master/Fronteira_%C3%B3tima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
tickers = ['VALE3.SA', 'TAEE11.SA']
periodo = '1y'

Baixando Cotações e Calculando Retornos


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

import plotly.graph_objects as go

from ipywidgets import interact

In [5]:
def fix_col_names(df):
  return ['IBOV' if col == '^BVSP' else col.rstrip('.SA') for col in df.columns]

In [6]:
prices = yf.download(tickers, period=periodo, rounding=True)['Adj Close']
prices.columns = fix_col_names(prices)
prices.dropna(inplace=True)
retorno = prices.pct_change().dropna()
ativos = retorno.columns.tolist()

[*********************100%***********************]  2 of 2 completed


In [7]:
prices.tail()

,TAEE11,VALE3
Date,,
2024-09-06 00:00:00+00:00,35.82,56.68
2024-09-09 00:00:00+00:00,35.67,56.68
2024-09-10 00:00:00+00:00,35.46,56.00
2024-09-11 00:00:00+00:00,35.30,57.59
2024-09-12 00:00:00+00:00,35.00,58.11


In [8]:
retorno.tail()

,TAEE11,VALE3
Date,,
2024-09-06 00:00:00+00:00,-0.000558,-0.012544
2024-09-09 00:00:00+00:00,-0.004188,0.000000
2024-09-10 00:00:00+00:00,-0.005887,-0.011997
2024-09-11 00:00:00+00:00,-0.004512,0.028393
2024-09-12 00:00:00+00:00,-0.008499,0.009029


Criando a Carteira